### Multi-Query Retriever Test

In [1]:
"""
< Multi-Query Retriever >

개떡같이 질문해도 찰떡같이 알아듣도록, 사용자 질문을 여러개의 유사 질문으로 재생성

ex)
사용자 질문 : B은행의 대출은 어때?
    재생성 질문 1 : B은행의 대출 금리는 어때?
    재생성 질문 2 : B은행의 대출 조건은 어떄?
    재생성 질문 3 : B은행의 대출 후기는 어때? 


(사용자 질문 + 재생성 질문들) --> VectorStore --> LLM
"""

'\n< Multi-Query Retriever >\n\n개떡같이 질문해도 찰떡같이 알아듣도록, 사용자 질문을 여러개의 유사 질문으로 재생성\n\nex)\n사용자 질문 : B은행의 대출은 어때?\n    재생성 질문 1 : B은행의 대출 금리는 어때?\n    재생성 질문 2 : B은행의 대출 조건은 어떄?\n    재생성 질문 3 : B은행의 대출 후기는 어때?\n\n\n(사용자 질문 + 재생성 질문들) --> VectorStore --> LLM\n'

In [2]:
# Libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Load Web Data
loader = WebBaseLoader("https://n.news.naver.com/mnews/article/015/0004995857")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
splits = text_splitter.split_documents(data)


# Embedding --> Vector DB
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {"normalize_embeddings":True}
ko_embedding = HuggingFaceEmbeddings(
    model_name = model_name,
    encode_kwargs = encode_kwargs
)
vectordb = Chroma.from_documents(documents=splits, embedding=ko_embedding)

C:\Users\jongh\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Libraries
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOllama

# llm --> retriever
llm = ChatOllama(model="wizardlm2")
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(),
    llm = llm
    #prompt = QUERY_PROMPT | 넣어줄 수 있음
)

In [6]:
# Set logging for the queries
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [9]:
question = "카카오톡의 이번 계획이 뭐야?"
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

## questions generated from Multi_query:
# INFO:langchain.retrievers.multi_query:Generated queries: 
# ['Original Question: 카카오톡의 이번 업데이트나 개선 계획은 어떻게 진행되고 있나요?', '',
#  'Alternative 1:', 'КакаоТалк의 최근에 발표된 기능 업데이트나 개발 로드맵에 대한 정보는 어디에서 확인할 수 있을까요?', '',
#  'Alternative 2:', '카카오톡의 최근 사용자 경험 향상 계획이나 새로운 기능 개발에 대한 공식 발표나 안내가 있는 곳은 어디인가요?', '',
#  'Alternative 3:', '카카오톡 앱의 최신 개발 도로맵이나 기능 업데이트 일정에 대한 자세한 정보를 제공하는 문서나 자료는 어떤 형태로 접근할 수 있을까요?']

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the current strategic plans that KakaoTalk is implementing for growth and development in the near future?', "2. Can you provide an overview of KakaoTalk's upcoming initiatives and features as outlined in their recent roadmap or announcement?", "3. I'm interested in learning about the direction KakaoTalk intends to take in its product and service offerings; could you detail any planned changes or updates they have disclosed?"]


7

In [8]:
# 질문관련(Question) - 7개의 유사문서가 추출되었음.
unique_docs

[Document(page_content='가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n\n\nAI 경쟁 언어모델에서 서비스로 넘어가는 단계기술 몰라도 쉽게 쓸 수 있는 카카오다운 서비스가 목표내정 이후 조직구조 개편에 초점...앞으로 프로세스와 문화까지 쇄신', metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/015/0004995857', 'title': "정신아 카카오 대표 “하반기 AI 접목한 서비스 내놓을 것”'"}),
 Document(page_content='이 기사를 본 이용자들이 함께 많이 본 기사, 해당 기사와 유사한 기사, 관심 기사 등을 자동 추천합니다\n닫기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n카카오 정신아 "연내 차별화된 AI 서비스 내놓겠다"\n언어모델 싸움에서 의미있는 서비스로 넘어가는 게임돼" \'데이터센터 안산\' 언론에 공개…"가장 안전한 데이터센터 건설 노력" 정신아 카카오 대표가 올해 안에 카카오의 차별화된 인공지능(AI) 서비스를 내놓겠다고 밝혔', metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/015/0004995857', 'title': "정신아 카카오 대표 “하반기 AI 접목한 서비스 내놓을 것”'"}),
 Document(page_content='정신아 카카오 대표는 “연내 카카오에 인공지능(AI)을 접목한 서비스를 내놓을 것”이라고 말했다.정 대표는 지난 11일 경기 안산시 한양대 에리카캠퍼스 내 카카오 데이터센터 안산에서 기자들과 만나 “카카오의 차별점은 월 4870만명이 이용하는 서비스라는 점”이라며 “AI 기술에 대해 잘 몰라도 이용자들이 쉽게 쓸 수 있도록 하는 게 우선적인 과제”라며 이같이 말했다. 관계를 기반으로 하는 카카오톡 서비스를 바탕으로 카카오